In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [21]:
# Read in data into a pandas dataframe and then show the contents of the dataframe
sub_file_path = '/home/levi/data/gnn_project/SEC_Filings_2023q2/sub.txt'
sub_df = pd.read_csv(sub_file_path, sep='\t', header=0, nrows=10)
cols = ['adsh', 'name', 'countryba', 'stprba', 'cityba', 'sic', 'ein', 'form', 'period', 'instance']
sub_df = sub_df[cols]
sub_df.head()


,adsh,name,countryba,stprba,cityba,sic,ein,form,period,instance
0,0000004457-23-000046,U-HAUL HOLDING CO /NV/,US,NV,RENO,7510,880106815,8-K,20230531,uhal-20230530_htm.xml
1,0000016732-23-000067,CAMPBELL SOUP CO,US,NJ,CAMDEN,2000,210419870,8-K,20230531,cpb-20230530_htm.xml
2,0000028823-23-000117,"DIEBOLD NIXDORF, INC",US,OH,HUDSON,3578,340183970,10-Q,20230331,dbd-20230331_htm.xml
3,0000047217-23-000035,HP INC,US,CA,PALO ALTO,3570,941081436,8-K,20230531,hpq-20230530_htm.xml
4,0000070318-23-000028,TENET HEALTHCARE CORP,US,TX,DALLAS,8062,952557091,8-K,20230531,thc-20230525_htm.xml


In [33]:
# Company lists
dow_list = ['axp', 'amgn', 'aapl', 'ba', 'cat', 'cvx', 'csco', 'ko', 'dow', 'gs', 'hd', 'hon', 'ibm', 'intc', 'jnj', 'jpm', 'mcd', 'mmm', 'mrk', 'msft', 'nke', 'pg', 'crm', 'trv', 'utx', 'vz', 'v', 'wba', 'wmt', 'dis']
# Create a list of the ticker symbols for the S&P 500 companies
sp500_list = ['a', 'aapl', 'abbv', 'abt', 'acn', 'adbe', 'aig', 'all', 'amgn', 'amzn', 'axp', 'ba', 'bac', 'biib', 'bk', 'bkng', 'blk', 'bmy', 'brk.b', 'bsx', 'cat', 'chtr', 'cl', 'cmcsa', 'cof', 'cop', 'cost', 'crm', 'csco', 'cvx', 'dhr', 'dis', 'dow', 'duke', 'emr', 'exc', 'f', 'fb', 'fdx', 'gd', 'ge', 'gild', 'gm', 'goog', 'googl', 'gs', 'hd', 'hon', 'ibm', 'intc', 'jnj', 'jpm', 'kmi', 'ko', 'lly', 'lmt', 'low', 'ma', 'mcd', 'mdlz', 'met', 'microsoft', 'mmm', 'mo', 'mrk', 'ms', 'msft', 'nflx', 'nke', 'nvda', 'orcl', 'pep', 'pfe', 'pg', 'pm', 'pypl', 'qcom', 'crm', 'sbux', 'slb', 'so', 'spg', 't', 'tgt', 'tmo', 'tpr', 'trv', 'tsla', 'txn', 'unh', 'unp', 'ups', 'usb', 'v', 'vz', 'wba', 'wfc', 'wmt', 'xom']
# Create a list of the ticker symbols for the NASDAQ 100 companies
nasdaq100_list = ['aapl', 'adbe', 'adi', 'adp', 'adsk', 'alxn', 'amat', 'amd', 'amgn', 'amzn', 'asml', 'atvi', 'avgo', 'bidu', 'biib', 'bkng', 'bmrn', 'bngo', 'cdns', 'celg', 'cern', 'chkp', 'chtr', 'cmcsa', 'cost', 'csco', 'csct', 'ctas', 'ctrp', 'ctsh', 'ctxs', 'dltr', 'ea', 'ebay', 'esrx', 'fast', 'fb', 'fisv', 'fox', 'foxa', 'gild', 'goog', 'googl', 'has', 'hsic', 'idxx', 'ilmn', 'incy', 'intc', 'intu', 'isrg', 'jd', 'khc', 'klac', 'lbtya', 'lbtym', 'liberty', 'lrcx', 'mar', 'mat', 'mchp', 'mdlz', 'meli', 'mnst', 'msft', 'mu', 'mxim', 'myl', 'nclh', 'nflx', 'ntap', 'ntes', 'nvda', 'nxpi', 'orly', 'payx', 'pcar', 'pypl', 'qcom', 'regn', 'rost', 'sbux', 'sirius', 'swks', 'symc', 'tmus', 'trip', 'tsco', 'tsem', 'txn', 'ulta', 'vrtx', 'vrsn', 'vrtx', 'wdc', 'wday', 'wltw', 'wynn', 'xlnx', 'xray', 'zm']



In [43]:
import requests
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=cvx&time_from20230701T0000=economy_macro&apikey=IMSWKZZ2ILUS0FKX'
r = requests.get(url)
data = r.json()

print(data)

{'items': '50', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'Nike Outperforms Among Dow Jones Stocks In October; Crocs Earnings On Deck', 'url': 'https://www.investors.com/news/nike-outperforms-among-dow-jones-stocks-in-october-crocs-earnings-on-deck/', 'time_published': '20231101T142148', 'authors': ["Investor's Business Daily", 'HARRISON MILLER'], 'summary': "Nike Outperforms Among Dow Jones Stocks In October. Crocs ... Investor's Business Daily ...", 'banner_image': 'https://www.investors.com/wp-content/uploads/2018/11/Stock-runnersLegs-01-adobe.jpg', 'source': 'Investors Business Daily', 'category_within_source': 'n/a', 'source_domain': 'www.investors.com', 'topics': [{'topic': 'Energy & Transportation', 'relevance_score': '0.25'}, {'topic': '

In [44]:
import pprint as pp
pp.pprint(data)

{'feed': [{'authors': ["Investor's Business Daily", 'HARRISON MILLER'],
           'banner_image': 'https://www.investors.com/wp-content/uploads/2018/11/Stock-runnersLegs-01-adobe.jpg',
           'category_within_source': 'n/a',
           'overall_sentiment_label': 'Somewhat-Bullish',
           'overall_sentiment_score': 0.232089,
           'source': 'Investors Business Daily',
           'source_domain': 'www.investors.com',
           'summary': 'Nike Outperforms Among Dow Jones Stocks In October. '
                      "Crocs ... Investor's Business Daily ...",
           'ticker_sentiment': [{'relevance_score': '0.147216',
                                 'ticker': 'WEYS',
                                 'ticker_sentiment_label': 'Neutral',
                                 'ticker_sentiment_score': '0.095257'},
                                {'relevance_score': '0.049322',
                                 'ticker': 'GOOG',
                                 'ticker_sentiment_l

In [23]:
num_file_path = '/home/levi/data/gnn_project/SEC_Filings_2023q2/num.txt'
num_df = pd.read_csv(num_file_path, sep='\t', header=0, nrows=10)
cols = ['adsh', 'tag', 'ddate', 'uom', 'value']
num_df = num_df[cols]
num_df.head()


,adsh,tag,ddate,uom,value
0,0000215466-23-000087,AccountsAndNotesReceivableNet,20221231,USD,4926000.0
1,0000215466-23-000087,AccountsAndNotesReceivableNet,20230331,USD,0.0
2,0001558370-23-008843,AccountsAndNotesReceivableNet,20221231,USD,56304000.0
3,0001558370-23-008843,AccountsAndNotesReceivableNet,20230331,USD,46241000.0
4,0001558370-23-009838,AccountsAndOtherReceivablesNetCurrent,20230331,USD,81729.0


In [32]:
tag_file_path = '/home/levi/data/gnn_project/SEC_Filings_2023q2/tag.txt'
tag_df = pd.read_csv(tag_file_path, sep='\t', header=0, nrows=100)
#cols = ['adsh', 'tag', 'ddate', 'uom', 'value']
#tag_df = tag_df[cols]
tag_df.head(50)

,tag,version,custom,abstract,datatype,iord,crdr,tlabel,doc
0,EntityCentralIndexKey,dei/2021,0,0,centralIndexKey,D,NaN,Entity Central Index Key,A unique 10-digit SEC-issued value to identify...
1,AmendmentFlag,dei/2021,0,0,boolean,D,NaN,Amendment Flag,Boolean flag that is true when the XBRL conten...
2,AccountingStandardsUpdate201613Member,us-gaap/2021,0,1,member,NaN,NaN,Accounting Standards Update 2016-13 [Member],Accounting Standards Update 2016-13 Financial ...
3,AccountingStandardsUpdate201602Member,us-gaap/2021,0,1,member,NaN,NaN,Accounting Standards Update 2016-02 [Member],Accounting Standards Update 2016-02 Leases (To...
4,EntityAddressAddressLine2,dei/2021,0,0,normalizedString,D,NaN,"Entity Address, Address Line Two",Address Line 2 such as Street or Suite number
5,TradingSymbol,dei/2021,0,0,tradingSymbol,D,NaN,Trading Symbol,Trading symbol of an instrument as listed on a...
6,StatementTable,us-gaap/2021,0,1,NaN,NaN,NaN,Statement [Table],"Schedule reflecting a Statement of Income, Sta..."
7,SecurityExchangeName,dei/2021,0,0,edgarExchangeCode,D,NaN,Security Exchange Name,Name of the Exchange on which a security is re...
8,NoTradingSymbolFlag,dei/2021,0,0,boolean,D,NaN,No Trading Symbol Flag,Boolean flag that is true only for a security ...
9,EntityRegistrantName,dei/2021,0,0,normalizedString,D,NaN,Entity Registrant Name,The exact name of the entity filing the report...


In [29]:
len(tag_df)

100

In [26]:
pre_file_path = '/home/levi/data/gnn_project/SEC_Filings_2023q2/pre.txt'
pre_df = pd.read_csv(pre_file_path, sep='\t', header=0, nrows=10)
#cols = ['adsh', 'tag', 'ddate', 'uom', 'value']
#tag_df = tag_df[cols]
pre_df.head(10)

,adsh,report,line,stmt,inpth,rfile,tag,version,plabel,negating
0,0000049600-23-000062,1,27,CP,0,H,AmendmentFlag,dei/2021,Amendment Flag,0
1,0000011544-23-000008,1,37,CP,0,H,AmendmentFlag,dei/2021,Amendment Flag,0
2,0000920371-23-000122,1,26,CP,0,H,AmendmentFlag,dei/2021,Amendment Flag,0
3,0000920371-23-000122,1,13,CP,0,H,CityAreaCode,dei/2021,City Area Code,0
4,0000011544-23-000008,1,21,CP,0,H,CityAreaCode,dei/2021,City Area Code,0
5,0000049600-23-000062,1,12,CP,0,H,CityAreaCode,dei/2021,City Area Code,0
6,0000008947-23-000042,6,3,BS,0,H,ClassOfStockDomain,us-gaap/2021,Class of Stock [Domain],0
7,0000008947-23-000042,7,3,BS,1,H,ClassOfStockDomain,us-gaap/2021,Class of Stock [Domain],0
8,0000008947-23-000042,9,10,EQ,0,H,ClassOfStockDomain,us-gaap/2021,Class of Stock [Domain],0
9,0000074260-23-000039,1,3,CP,0,H,ClassOfStockDomain,us-gaap/2021,Class of Stock [Domain],0


In [15]:
df.columns

Index(['adsh', 'cik', 'name', 'sic', 'countryba', 'stprba', 'cityba', 'zipba',
       'bas1', 'bas2', 'baph', 'countryma', 'stprma', 'cityma', 'zipma',
       'mas1', 'mas2', 'countryinc', 'stprinc', 'ein', 'former', 'changed',
       'afs', 'wksi', 'fye', 'form', 'period', 'fy', 'fp', 'filed', 'accepted',
       'prevrpt', 'detail', 'instance', 'nciks', 'aciks'],
      dtype='object')

In [ ]:


# Fake data feature:
# quarter of the year [range(4)] (time of year dependency)
# market capitalization $B (company size dependency)
# industry [range(11)] of the SP500 sectors (type of company dependency)
# PE ratio (expected growth dependency)
# Analyst rating [range(3)] sell=0, hold=1, buy=2
fake_features = [
    (np.random.randint(0, 4),
      np.random.randint(0, 100), 
      np.random.randint(0, 11), 
      np.random.randint(0, 100), 
      np.random.randint(0, 3)) for _ in range(10)
]


In [2]:
fake_features

[(3, 4, 10, 8, 2),
 (2, 13, 0, 56, 0),
 (2, 47, 6, 47, 0),
 (1, 9, 7, 60, 1),
 (0, 53, 2, 25, 1),
 (2, 70, 7, 80, 2),
 (3, 20, 10, 67, 0),
 (0, 3, 2, 98, 2),
 (0, 91, 8, 52, 2),
 (2, 20, 4, 62, 0)]

In [9]:
import torch
from torch_geometric.data import Data

# Nodes (companies) and their features
# x = torch.tensor([feature_vector_for_company_1, feature_vector_for_company_2, ...], dtype=torch.float)
x = torch.tensor(fake_features, dtype=torch.float)

# Edge connections 
# edge_index = torch.tensor([
#     [source_node_1, source_node_2, ...],
#     [target_node_1, target_node_2, ...],
# ], dtype=torch.long)
edge_index = torch.tensor([
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    [1, 0, 3, 2, 5, 4, 7, 6, 9, 8],
], dtype=torch.long)

# Labels for nodes (whether stock price will go up (1) or down (0))
#y = torch.tensor([label_for_company_1, label_for_company_2, ...], dtype=torch.float)
y = torch.tensor([
    np.random.randint(0, 2) for _ in range(10)
], dtype=torch.long)

data = Data(x=x, edge_index=edge_index, y=y)


In [11]:
# Inspect the data shape
print(f"Node data shape: {x.shape}")
print(f"Edge data shape: {edge_index.shape}")
print(f"Label data shape: {y.shape}")   
data.num_classes = 1
data.num_nodes = 10


Node data shape: torch.Size([10, 5])
Edge data shape: torch.Size([2, 10])
Label data shape: torch.Size([10])


In [12]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class StockGNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 3)  # 5 -> 3
        self.conv2 = GCNConv(3, data.num_classes) # 3 -> 1

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = StockGNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()

IndexError: Target 1 is out of bounds.